In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
#sys.path.append('shallow')

# Imports

In [ ]:
#export
import time
from functools import partial

import torch
from torch.utils.tensorboard import SummaryWriter

from shallow import utils, meters
#from utils import GetAttr

# Code

In [ ]:
#export 
class CancelFitException(Exception): pass

class Callback(utils.GetAttr): 
    _default='learner'
    def __init__(self, *args, **kwargs):
        utils.store_attr(self, locals())
    
class ParamSchedulerCB(Callback):
    def __init__(self, phase, pname, sched_func):
        self.pname, self.sched_func = pname, sched_func
        setattr(self, phase, self.set_param)
        
    def set_param(self):
        setattr(self.learner, self.pname, self.sched_func(self.epoch/self.n_epochs))
    
class SetupLearnerCB(Callback):
    def before_batch(self):
        xb,yb = to_device(self.batch)
        self.learner.batch = tfm_x(xb),yb

    def before_fit(self): self.model.cuda()

class TrackResultsCB(Callback):
    def before_epoch(self): self.accs,self.losses,self.ns = [],[],[]
        
    def after_epoch(self):
        n = sum(self.ns)
        print(self.epoch, self.model.training,
              sum(self.losses).item()/n, sum(self.accs).item()/n)
        
    def after_batch(self):
        xb, yb = self.batch
        acc = (self.preds.argmax(dim=1)==yb).float().sum()
        self.accs.append(acc)
        n = len(xb)
        self.losses.append(self.loss*n)
        self.ns.append(n)

class LRFinderCB(Callback):
    def before_fit(self):
        self.losses,self.lrs = [],[]
        self.learner.lr = 1e-6
            
    def before_batch(self):
        if not self.model.training: return
        self.learner.lr *= 1.2
        print(self.lr)
        
    def after_batch(self):
        if not self.model.training: return
        if self.lr > 1 or torch.isnan(self.loss): raise CancelFitException
        self.losses.append(self.loss.item())
        self.lrs.append(self.lr)
        
        
class TimerCB(Callback):
    def __init__(self, Timer=None, logger=None):
        self.logger = logger
        self.perc=90
        if Timer is None: Timer = meters.StopwatchMeter
        self.batch_timer = Timer()
        self.epoch_timer = Timer()
    
    def before_batch(self): self.batch_timer.start()
    def before_epoch(self): self.epoch_timer.start()
    def after_batch(self):  self.batch_timer.stop()

    def after_epoch(self):
        self.epoch_timer.stop()
        bs, es = self.learner.dl.batch_size, len(self.learner.dl)
        if self.logger is not None: self.logger.info(f'\tEpoch {self.epoch}: {self.epoch_timer.last: .3f} s,'+
                                                     f'{bs * es/self.epoch_timer.last: .3f} im/s; '+
                                                     f'batch {self.batch_timer.avg: .3f} s'  
                                                     )    
        #if self.logger is not None: self.logger.info(f'\t\tBatch : ')
        self.batch_timer.reset()
    
    def after_fit(self):
        if self.logger is not None:
            et = self.epoch_timer
            em = et.avg
            estd = ((et.p(self.perc) - em) + (em - et.p(1-self.perc))) / 2
            self.logger.info(f'\tEpoch average time: {em: .3f} +- {estd: .3f} s')
        
        
class CheckpointCB(Callback):
    def __init__(self, save_path, save_step=None):
        utils.store_attr(self, locals())
        self.pct_counter = None if isinstance(self.save_step, int) else self.save_step
        
    def after_epoch(self):
        save = False
        if self.epoch == self.n_epochs - 1: save=True
        elif isinstance(self.save_step, int): save = self.save_step % self.epoch == 0 
        else:
            if self.epoch / self.n_epochs > self.pct_counter:
                save = True
                self.pct_counter += self.save_step
        
        if save:
            torch.save({
                                'epoch': self.epoch,
                                'loss': self.loss,
                                'model_state':self.model.state_dict(),
                                'opt_state':self.opt.state_dict(),                    
                            }, str(self.save_path / f'e{self.epoch}_t{self.n_epochs}_1e4l{int(1e4*self.loss)}.pth'))

In [ ]:
#export
class TensorBoardCB(TrackResultsCB):
    def __init__(self, writer, logger=None, save_images=False, step=1, metrics={'totals':['total_loss', 'accuracy']}):
        utils.store_attr(self, locals())
        
    def after_epoch(self):
        n = sum(self.ns)
        self.accuracy = sum(self.accs).item()/n
        self.total_loss = sum(self.losses).item()/n
        for category, metrics in self.metrics.items():
            for metric in metrics:
                #if self.logger is not None: self.logger.debug(f"{category + '/' + metric, getattr(self, metric), self.epoch}")
                self.writer.add_scalar(category + '/' + metric, getattr(self, metric), self.epoch)
        
        if self.save_images and (self.epoch % self.step) == 0:
            count = 4
            images, masks, preds = self.batch[0], self.batch[1], self.preds
            masks = masks.repeat(1,3,1,1)
            preds = torch.sigmoid(preds.repeat(1,3,1,1))
            images = images[:count]
            masks = masks[:count]
            preds = preds[:count]
            
            summary_image = torch.cat([255. * images.detach().cpu(), masks.detach().cpu(), preds.detach().cpu()], 0)
            #self.logger.debug(f"{summary_image.shape}, {images.shape}, {masks.shape}")
            grid = torchvision.utils.make_grid(summary_image, nrow=4, pad_value=1)
            self.writer.add_image('images', grid, self.epoch)
        
        self.writer.flush()
        
#             if self.track_weight:
#                 for k, v in self.learn.model.state_dict().items():
#                     self.writer.add_histogram(k, v, kwargs['epoch']+self.delta_epochs)      
            
                  
#     def on_backward_end(self, **kwargs):
#         if self.track_grad and 'last_metrics' in kwargs and kwargs['last_metrics'][0] != None and kwargs['epoch'] > self.last_epoch_backward:
#             self.last_epoch_backward = kwargs['epoch']
#             keys = list(self.learn.model.state_dict().keys())
#             indice = 0
#             for i, param in enumerate(self.learn.model.parameters()):
#                 grad = param.grad
#                 if grad is not None:
#                     self.writer.add_histogram('back'+'.'+keys[i], grad, kwargs['epoch']+self.delta_epochs)
#                     i += 1

In [ ]:
#export
def append_stats(hook, mod, inp, outp, bins=100, vmin=0, vmax=0):
    if not hasattr(hook,'stats'): hook.stats = ([],[],[])
    means,stds,hists = hook.stats
    means.append(outp.data.mean().cpu())
    stds .append(outp.data.std().cpu())
    hists.append(outp.data.cpu().histc(bins,vmin,vmax))
    
class Hook():
    def __init__(self, m, f): self.hook = m.register_forward_hook(partial(f, self))
    def remove(self): self.hook.remove()
    def __del__(self): self.remove()
        
class Hooks(utils.ListContainer):
    def __init__(self, ms, f): super().__init__([Hook(m, f) for m in ms])
    def __enter__(self, *args): return self
    def __exit__ (self, *args): self.remove()
    def __del__(self): self.remove()

    def __delitem__(self, i):
        self[i].remove()
        super().__delitem__(i)
        
    def remove(self):
        for h in self: h.remove()

# Tests

### Param sched

In [ ]:
import torch
import matplotlib.pyplot as plt

from shallow import schedulers

In [ ]:
class Learner:
    def __init__(self, cbs):
        self.cbs = cbs
        self.lr = -1
        self.epochs = 100
        self.n_epochs = 13
        for c in self.cbs: c.learner=self
    
    def t(self):
        self('before_epoch')
        
    def tt(self):
        self('bla_bla')
        
    def __call__(self, name):
        for cb in self.cbs: getattr(cb, name, utils.noop())()

In [ ]:

p = ParamScheduler('before_epoch', 'lr', schedulers.sched_lin(0,.1))

In [ ]:
l = Learner([p])
l.n_epochs = 88 # out of 100
l.t()
l.lr

### Hooks

## TB

In [ ]:
writer = SummaryWriter(comment='Demo')
mycallback = partial(TensorBoardCB, writer, track_weight=True, track_grad=True, metric_names=['val loss', 'accuracy'])
